In [1]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 4)

from modules.Z3.model import * 

In [2]:
FI = 300 # mm
TW = 10 # mm
H = 50 # m
M = 1000 # kg
q = 1.5
ag = 0.63
vl_tvary = 3
hmoty = 4

Lokalita - Bratislava\
Podlozie - C\
Spektrum - 2

In [3]:
# jednotkova sila posobiaca v bode 2
F_2 = [32365.2,     # bod 2
       80861.9,     # bod 3
       129359.0,    # bod 4
       177855.0     # bod 5
       ]
# jednotkova sila posobiaca v bode 3
F_3 = [80861.9,
       258717.0,
       452704.0,
       646691.0,
       ]
# jednotkova sila posobiaca v bode 4
F_4 = [129359.0,
       452704.0,
       873043.0,
       1309510.0,
       ]
# jednotkova sila posobiaca v bode 5
F_5 = [177855.0,
       646691.0,
       1309510.0,
       2069330.0,
       ]

In [4]:
# frekvencie
f = [0.071, #1 tvar
     0.436, #2 tvar
     1.200] #3 tvar


In [5]:
T = []
i = 0
for frekvencia in f:
    i += 1
    perioda = (1/frekvencia)
    print(f"Perioda pre vlastny tvar {i}: {perioda:.4f} s")
    T.append(perioda)


Perioda pre vlastny tvar 1: 14.0845 s
Perioda pre vlastny tvar 2: 2.2936 s
Perioda pre vlastny tvar 3: 0.8333 s


In [7]:
def ux(F):
    # reverse list
    F_x = np.array([F])
    F_x_flipped = np.flip(F_x, axis=1)
    return F_x_flipped.flatten()


In [8]:
V0 = [F_2, F_3, F_4, F_5]

In [9]:
V00 = []
for i in V0:
    V00.append(ux(i))

In [10]:
V00

[array([177855. , 129359. ,  80861.9,  32365.2]),
 array([646691. , 452704. , 258717. ,  80861.9]),
 array([1309510.,  873043.,  452704.,  129359.]),
 array([2069330., 1309510.,  646691.,  177855.])]

In [11]:
F_5 = ux(F_5)
F_4 = ux(F_4)
F_3 = ux(F_3)
F_2 = ux(F_2)

In [12]:
delta = pd.DataFrame([F_5, F_4, F_3, F_2], columns=["F_2", "F_3", "F_4", "F_5"])/(10**8)
delta

,F_2,F_3,F_4,F_5
0,0.0207,0.0131,0.0065,0.0018
1,0.0131,0.0087,0.0045,0.0013
2,0.0065,0.0045,0.0026,0.0008
3,0.0018,0.0013,0.0008,0.0003


spektrum pruznej odozvy

In [13]:
def spektrum_parametre_2(podlozie):
    if podlozie == "A":
        S = 1
        T_B = 0.05
        T_C = 0.25
        T_D = 1.2

    if podlozie == "B":
        S = 1.35
        T_B = 0.05
        T_C = 0.25
        T_D = 1.2
    
    if podlozie == "C":
        S = 1.5
        T_B = 0.1
        T_C = 0.25
        T_D = 1.2

    if podlozie == "D":
        S = 1.8
        T_B = 0.1
        T_C = 0.3
        T_D = 1.2

    if podlozie == "E":
        S = 1.6
        T_B = 0.05
        T_C = 0.25
        T_D = 1.2

    parametre_podlozia = {
        "S" : S,
        "T_B": T_B,
        "T_C": T_C,
        "T_D": T_D,
    }

    return parametre_podlozia


In [14]:
def matica_hmotnosti(m):
    np.diag([m, m, m, m])

In [15]:
def sd(parametre, T0, ag, q):
    # navrhove spektrum sd
    S = parametre["S"]
    T_B = parametre["T_B"]
    T_C = parametre["T_C"]
    T_D = parametre["T_D"]

    beta = 0.2 # narodna priloha smie stanovit hodnotu beta. Doporucena hodnota je 0.2

    if 0 <= T0 < T_B:
        sd = ag*S*(2/3+T0/T_B*(2.5/q-2/3))
    elif T_B <= T0 < T_C:
        sd = ag*S*(2.5/q)
    elif T_C <= T0 < T_D:
        sd = ag*S*2.5/q*(T_C/T0)
        if sd < ag*beta:
            sd = ag*beta
    elif T_D <= T0:
        sd = ag*S*2.5/q*(T_C*T_D/T0**2)
        if sd < ag*beta:
            sd = ag*beta
    print(f"sd = {sd}")
    return sd    


In [16]:
Sd_1 = sd(spektrum_parametre_2("C"), T0=T_1, ag=ag, q=q)
Sd_2 = sd(spektrum_parametre_2("C"), T0=T_2, ag=ag, q=q)
Sd_3 = sd(spektrum_parametre_2("C"), T0=T_3, ag=ag, q=q)

sd = 0.126
sd = 0.126
sd = 0.4726890756302522


In [17]:
M_tot = 3*M
lambda_ = 1 # !Preco??

celkova smykova sila pri zaklade

In [18]:
def Fb(M_tot, lambda_, sd):
    Fb =  M_tot*lambda_*sd
    print(f"Fb = {Fb}")
    return Fb

In [19]:
FB_1 = Fb(M_tot, lambda_, Sd_1)
FB_2 = Fb(M_tot, lambda_, Sd_2)
FB_3 = Fb(M_tot, lambda_, Sd_3)

Fb = 378.0
Fb = 378.0
Fb = 1418.0672268907565


vysledne sily

In [20]:
def Si(delta, M, index):
    # index je index riadku v matici delta
    Si = delta.iloc[[index]].T*M
    print(f"Si \n {Si}")
    print("---")
    return Si

In [21]:
S_1 = Si(delta, M, 0)
S_2 = Si(delta, M, 1)
S_3 = Si(delta, M, 2)

Si 
            0
F_2  20.6933
F_3  13.0951
F_4   6.4669
F_5   1.7786
---
Si 
            1
F_2  13.0951
F_3   8.7304
F_4   4.5270
F_5   1.2936
---
Si 
           2
F_2  6.4669
F_3  4.5270
F_4  2.5872
F_5  0.8086
---


In [22]:
def Fi(Si_, Fb_, M):
    Si_tot = Si_.sum()
    Fi =  Fb_ * (Si_*M)/(Si_tot * M)
    print(f"Fi \n {Fi}")
    print("---")
    return Fi


In [23]:
Fi_1 = Fi(S_1, FB_1, M)
Fi_2 = Fi(S_2, FB_2, M)
Fi_3 = Fi(S_3, FB_3, M)

Fi 
             0
F_2  186.0897
F_3  117.7610
F_4   58.1553
F_5   15.9941
---
Fi 
             1
F_2  179.0465
F_3  119.3693
F_4   61.8972
F_5   17.6870
---
Fi 
             2
F_2  637.2953
F_3  446.1267
F_4  254.9581
F_5   79.6871
---


In [24]:
def H_vector(H):
    H_vect =  np.array([[H/4], [2*H/4], [3*H/4], [H]])
    return H_vect

In [25]:
def Moment(Fi_, H_vector):
    M = Fi_.values*H_vector
    print(f"Moment [kN*m]\n {M}")
    print("---")
    return pd.DataFrame(M)

In [26]:
M_1 = Moment(Fi_1, H_vector(H))
M_2 = Moment(Fi_2, H_vector(H))
M_3 = Moment(Fi_3, H_vector(H))

Moment [kN*m]
 [[2326.12095344]
 [2944.02405584]
 [2180.82396549]
 [ 799.70278723]]
---
Moment [kN*m]
 [[2238.08107527]
 [2984.23229483]
 [2321.14666196]
 [ 884.3488933 ]]
---
Moment [kN*m]
 [[ 7966.19133454]
 [11153.16799496]
 [ 9560.92998124]
 [ 3984.35337479]]
---
